# Split file into clean subfiles, create dirty files with synthetic errors
## Author: David Schanzer
## Student ID: 82329622
### Subject: 32933 Research Project
### Session: Spring 2023
### Assessment 1: Prototype Research
### Supervisor: Dr Amara Atif
### This Jupyter Notebook takes a single large multi-year CSV file that contains "clean" radiotherapy treatment data and creates individual files for each month, fortnight and week. For each of these "clean" files, it also creates an equivalent "dirty" file that contains between 1% and 80% of records with a synthetic error, which can one of six different types of error.

Import all required Python modules

In [10]:
import datetime
import math
import numbers
import random
from pathlib import Path
import numpy as np
import pandas as pd

**_is_number:_** Define a method to determine whether a string contains a number.

In [11]:
def is_number(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

**rmtree:_** Define a method to remove all files and folders at and below the given pathname.

In [12]:
def rmtree(f: Path):
    if f.is_file():
        f.unlink()
    else:
        for child in f.iterdir():
            rmtree(child)
        f.rmdir()

**create_folder:_** Define a method to create an empty folder at the specified pathname, deleting existing files if necessary.

In [13]:
def create_folder(baseFolder, folderName):
    folderPath = baseFolder + folderName
    folderPathObj = Path(folderPath)
    if folderPathObj.exists():  # If folder already exists
        rmtree(folderPathObj)   # then recursively delete all files and folders

    folderPathObj.mkdir()       # Create a new empty folder

**gaussian_noise:_** Define a method to take a numeric value and add "noise", varying the value within the given mean and standard deviation.

In [14]:
def gaussian_noise(value, mean, std):
    noise = np.random.normal(mean, std)
    value_noisy = float(value) + noise
    return value_noisy

**generate_clean_files:_** Define a method to generate all required "clean" files from a given dataframe.

In [15]:
def generate_clean_files(df, sliceBy, cleanFileNamePrefix, cleanPath):
    # Add columns for year, month and week
    df['tSiteStartDateYear'] = df['tSiteStartDate'].dt.year
    df['tSiteStartDateMonth'] = df['tSiteStartDate'].dt.month
    df['tSiteStartDateWeek'] = df['tSiteStartDate'].apply(lambda x: x.isocalendar()[1])
    df['tSiteStartDateFortnight'] = df['tSiteStartDate'].apply(lambda x: np.ceil(x.isocalendar()[1] / 2))

    if sliceBy == 'Week':          # Slicing by week
        byGroup = df.groupby(['tSiteStartDateYear', 'tSiteStartDateWeek'])
    elif sliceBy == 'Fortnight':   # Slicing by fortnight
        byGroup = df.groupby(['tSiteStartDateYear', 'tSiteStartDateFortnight'])
    else:                          # Slicing by month
        byGroup = df.groupby(['tSiteStartDateYear', 'tSiteStartDateMonth'])

    iterator = iter(byGroup)       # Create an iterator for the groupby object

    # Slice the input file into separate files in the clean folder
    index = 0
    while index < len(byGroup):
        group, frame = next(iterator)   # Return the next item from the iterator
        year = group[0]                 # The first element of the tuple is the year
        period_number = int(group[1])   # The second element of the tuple is the month or fortnight or week number
        cleanFileName = (cleanFileNamePrefix + str(index + 1).zfill(3) + '_' + str(year) + '_' + sliceBy +
                         str(period_number).zfill(2) + '.csv')

        # Drop the extra columns that we added from this frame before it's saved
        frame = frame.drop(['tSiteStartDateYear', 'tSiteStartDateMonth', 'tSiteStartDateFortnight',
                            'tSiteStartDateWeek'], axis=1)
        frame.sort_values(by=['tSiteStartDate'], inplace=True)      # Sort the frame by tSiteStartDate

        # Replace NaN values with 'NULL', as this is how empty cells were indicated in the input file
        frame.replace(np.nan, 'NULL', inplace = True)
        # Save this frame to a CSV file
        frame.to_csv(cleanPath + '/' + cleanFileName, index = False, encoding = 'cp1252')

        index += 1

**injectError:_** Define a method to inject a given error type into a given line of data passed as a list.

In [16]:
def injectError(dirtyList, randomErrorType, errorColumn, errorColumn2, meanContinuousNumericValue, typoEveryNthChar):
    if randomErrorType == 1:     # Explicit missing values (empty cells)
        # Just set this column to NULL since this is how empty columns are set in this dataset
        dirtyList[errorColumn] = 'NULL'
        
    elif randomErrorType == 2:   # Implicit missing values (logically empty, 99999 for numerics or NONE for text)
        if is_number(dirtyList[errorColumn]):
            dirtyList[errorColumn] = '99999'   # Set numeric columns to 99999 to indicate unknown
        else:
            dirtyList[errorColumn] = 'NONE'    # Set non-numeric columns to NONE to indicate unknown
            
    elif randomErrorType == 3:   # Numeric anomalies (unexpectedly high or low numeric values)
        newValue = dirtyList[errorColumn]
        
        if pd.isnull(newValue):
            newValue = meanContinuousNumericValue   # Since there is a null in this cell, use this column's mean
                                                    # as a starting point (before applying Gaussian noise)
            
        if is_number(newValue):
            randomStDevs = np.random.randint(2,5)   # Choose a random number of standard deviations between 2 and 5
            # Apply Gaussian noise with this number of SDs
            dirtyList[errorColumn] = str(np.round(gaussian_noise(value = newValue, mean = 0, std = randomStDevs), 1))
        
    elif randomErrorType == 4:   # Swapped numeric fields (swapping values from one numeric attribute to another)
        swapValue               = dirtyList[errorColumn]
        dirtyList[errorColumn]  = dirtyList[errorColumn2]
        dirtyList[errorColumn2] = swapValue
        
    elif randomErrorType == 5:   # Swapped textual fields (swapping values from one textual attribute to another)
        swapValue               = dirtyList[errorColumn]
        dirtyList[errorColumn]  = dirtyList[errorColumn2]
        dirtyList[errorColumn2] = swapValue
        
    elif randomErrorType == 6:   # Typographic errors (randomly replace a fraction of letters in textual attributes
                                 # with adjacent letters on a QWERTY keyboard)
        # Create dictionary of adjacent keys on a QWERTY keyboard
        adjacentLetters = {
            '1' : ('2','Q'),
            '2' : ('1','3','Q','W'),
            '3' : ('2','4','W','E'),
            '4' : ('3','5','E','R'),
            '5' : ('4','6','R','T'),
            '6' : ('5','7','T','Y'),
            '7' : ('6','8','Y','U'),
            '8' : ('7','9','U','I'),
            '9' : ('8','0','I','O'),
            '0' : ('9','O','P'),
            'Q' : ('1','2','W','A'),
            'W' : ('2','3','Q','E','A','S'),
            'E' : ('3','4','W','R','S','D'),
            'R' : ('4','5','E','T','D','F'),
            'T' : ('5','6','R','Y','F','G'),
            'Y' : ('6','7','T','U','G','H'),
            'U' : ('7','8','Y','I','H','J'),
            'I' : ('8','9','U','O','J','K'),
            'O' : ('9','0','I','P','K','L'),
            'P' : ('0','O','L'),
            'A' : ('Q','W','S','Z'),
            'S' : ('W','E','A','D','Z','X'),
            'D' : ('E','R','S','F','X','C'),
            'F' : ('R','T','D','G','C','V'),
            'G' : ('T','Y','F','H','V','B'),
            'H' : ('Y','U','G','J','B','N'),
            'J' : ('U','I','H','K','N','M'),
            'K' : ('I','O','J','L','M'),
            'L' : ('O','P','K'),
            'Z' : ('A','S','X'),
            'X' : ('S','D','Z','C'),
            'C' : ('D','F','X','V'),
            'V' : ('F','G','C','B'),
            'B' : ('G','H','V','N'),
            'N' : ('H','J','B','M'),
            'M' : ('J','K','N')
        }
        string = dirtyList[errorColumn]
        if pd.isnull(string):
            string = ''
        stringList = list(string)
        
        for element in range(0, len(stringList), typoEveryNthChar):
            char = stringList[element]
            
            if char.isalpha() or char.isnumeric():      # Only replace from the dictionary if a number or a letter
                upperChar = char.upper()                # Retrieve this character and convert to upper-case
                replacementCharList = adjacentLetters[upperChar]   # Look up the replacement char in the dictionary
                randomListElement = random.randint(0, len(replacementCharList) - 1)   # Random element
                replacementChar = replacementCharList[randomListElement]

                if not char.isupper():                  # If the original character was lower-case
                    replacementChar = replacementChar.lower()   # then make replacement character lower-case too

                stringList[element] = replacementChar   # Set the replacement character in the string
                
        dirtyList[errorColumn] = "".join(stringList)    # Replace the string in the dirtyList element
    else:
        print('Error: randomErrorType set to ' + str(randomErrorType))

    return dirtyList

**generate_dirty_files:_** Define a method to generate all required "dirty" files from the "clean" files in a given pathname.

In [17]:
def generate_dirty_files(cleanPath, cleanName, dirtyPath, dirtyName, dtypes, parse_dates, errorMagnitudes,
                         typoEveryNthChar, continuousNumericMeans, continuousNumericColumns):
    
    # Iterate through each of the clean files to create the equivalent dirty file
    files = Path(cleanPath).glob('Radiotherapy*')
    dlist = list(dtypes)
    columnNames = ','.join(dlist) + '\n'
    
    for file in sorted(files):
        # Load the next clean file into a dataframe
        cleanDF = pd.read_csv(file, sep = ',', dtype = dtypes, parse_dates = parse_dates, encoding = 'cp1252')

        fileErrorMagnitude = errorMagnitudes[random.randint(0,9)]                  # Randomly % of error records
        targetErrorCount = math.ceil(fileErrorMagnitude / 100 * cleanDF.shape[0])  # Number of records (rounded up)
    
        # Inject one of the following types of synthetic error:
        # 1. Explicit missing values (empty cells)
        # 2. Implicit missing values (cells that are logically empty but contain 99999 for numeric fields or NONE
        #    for textual fields)
        # 3. Numeric anomalies (unexpectedly high or low numeric values)
        # 4. Swapped numeric fields (swapping values from one numeric attribute to another)
        # 5. Swapped textual fields (swapping values from one textual attribute to another)
        # 6. Typographic errors (randomly replace a fraction of letters in textual attributes with adjacent letters
        #    on a QWERTY keyboard)

        randomErrorType = random.randint(1,6)           # Choose a random number between 1 and 6
        #randomErrorType = 6
        numberColumns = cleanDF.shape[1]                # Count the number of columns in the header row
        errorColumn2 = np.NaN                           # Initialise 2nd error column to NaN since not always
                                                        # needed
        meanContinuousNumericValue = np.NaN             # Initialise mean continuuous numeric value to NaN

        # List the column elements that are never null, are numeric, non-numeric respectively.
        #
        # Non-null column numbers below are:
        # EpisodeID, PatientID, CaseID, tFirstGivenName, tSurname, tDateOfBirth, tTreatmentStartDate, tSource,
        # TreatmentYear, Snapshot, AgeAtTreatment, PatientID.Derived, AddressID.Derived, AddressID, CaseID.Derived,
        # RecordID, SiteNameBoost, SiteNameLymphNode
        nonNullColumns = [0, 3, 4, 9, 11, 13, 53, 68, 72, 73, 75, 82, 107, 108, 109, 110, 112, 113]
        # Non-null numeric column numbers below are:
        # EpisodeID, PatientID, CaseID, TreatmentYear, AgeAtTreatment, AddressID, RecordID
        nonNullNumericColumns = [0, 3, 4, 72, 75, 108, 110]           
        # Non-null non-numeric column numbers below are:
        # tFirstGivenName, tSurname, tDateOfBirth, tTreatmentStartDate, tSource, Snapshot, PatientID.Derived,
        # AddressID.Derived, CaseID.Derived, SiteNameBoost, SiteNameLymphNode
        nonNullNonNumericColumns = [9, 11, 13, 53, 68, 73, 82, 107, 109, 112, 113]                   
        
        if randomErrorType == 1:     # Explicit missing values (empty cells)
            # Select a non-empty column by picking a random integer between 0 and len(nonNullColumns) - 1
            errorColumn = nonNullColumns[random.randint(0, len(nonNullColumns) - 1)]
                      
        elif randomErrorType == 2:   # Implicit missing values (cells that are logically empty but contain 99999
                                     # for numeric fields or NONE for textual fields)
            # Select a non-empty column by picking a random integer between 0 and len(nonNullColumns) - 1
            errorColumn = nonNullColumns[random.randint(0, len(nonNullColumns) - 1)]
                    
        elif randomErrorType == 3:   # Numeric anomalies (unexpectedly high or low numeric values)
            # Select a continuous numeric column by picking a random integer between 0 and
            # len(continuousNumericColumns) - 1
            errorColumn = continuousNumericColumns[random.randint(0, len(continuousNumericColumns) - 1)]
            meanContinuousNumericValue = continuousNumericMeans[str(errorColumn)]
                    
        elif randomErrorType == 4:   # Swapped numeric fields (swapping values from one numeric attribute to
                                     # another)
            # Select a numeric column by picking a random integer between 0 and len(nonNullNumericColumns) - 1
            errorColumn = nonNullNumericColumns[random.randint(0, len(nonNullNumericColumns) - 1)]
                    
            # Find a second numeric column with which we can swap values
            while True:
                # Pick a random integer between 0 and len(nonNullNumericColumns) - 1
                errorColumn2 = nonNullNumericColumns[random.randint(0, len(nonNullNumericColumns) - 1)]
                if errorColumn2 != errorColumn:   # Ensure that it's a different column
                    break
                    
        elif randomErrorType == 5:   # Swapped textual fields (swapping values from one textual attribute to
                                     # another)
            # Select a non-numeric column by picking a random integer between 0 and
            # len(nonNullNonNumericColumns) - 1
            errorColumn = nonNullNonNumericColumns[random.randint(0, len(nonNullNonNumericColumns) - 1)]
                    
            # Select a second non-numeric column with which we can swap values
            while True:
                # Pick a random integer between 0 and len(nonNullNonNumericColumns) - 1
                errorColumn2 = nonNullNonNumericColumns[random.randint(0, len(nonNullNonNumericColumns) - 1)]
                if errorColumn2 != errorColumn:   # Ensure that it's a different column
                    break
                    
        elif randomErrorType == 6:   # Typographic errors (randomly replace a fraction of letters in textual
                                     # attributes with adjacent letters on a QWERTY keyboard)
            # Select a non-numeric column by picking a random integer between 0 and
            # len(nonNullNonNumericColumns) - 1
            errorColumn = nonNullNonNumericColumns[random.randint(0, len(nonNullNonNumericColumns) - 1)]
        else:
            print('Error: randomErrorType set to ' + str(randomErrorType))

        # Start with the dirty file name matching the clean file name (and 'clean' replaced with 'dirty')
        dirtyFileName = file.name.replace(cleanName, dirtyName)
        
        # Add suffixes for the percentage of records in error and the type of error injected into this file
        dirtyFileName = dirtyFileName[:-4] + '_' + str(fileErrorMagnitude) + '%_error_' + str(randomErrorType) +
                        '_in_'
        if randomErrorType == 4 or randomErrorType == 5:
            dirtyFileName += 'columns_' + dlist[errorColumn] + '_and_' + dlist[errorColumn2]
        else:
            dirtyFileName += 'column_' + dlist[errorColumn]
        
        dirtyFileName += '.csv'
        
        dirtyFile = open(dirtyPath + '/' + dirtyFileName, 'w')   # Open the next dirty file for writing
        dirtyFile.write(columnNames)                             # Write the column names as the first row
        errorCount = 0

        #debugFileName = file.name.replace(cleanName, 'debug')
        #debugFile = open(dirtyPath + '/' + debugFileName, 'w')
        for cleanTuple in cleanDF.itertuples():                  # For each row in the clean dataframe
            dirtyList = list(cleanTuple[1:])                     # Convert clean named tuple to dirty list, so that
                                                                 # it can be updated if needed, and remove index

            if errorCount < targetErrorCount:                    # More errors need to be injected
                # Inject an error into this line
                dirtyList = injectError(dirtyList, randomErrorType, errorColumn, errorColumn2,
                                        meanContinuousNumericValue, typoEveryNthChar)
                errorCount += 1

            dirtyTuple = tuple(dirtyList)                        # Convert list back to tuple
            dirtyDF = pd.DataFrame(dirtyTuple)                   # Convert tuple to dataframe
            # Convert dataframe to comma-separated string (elements separated by comma and space)
            dirtyStr = dirtyDF.to_csv(header = False, index = False, na_rep = 'NULL', encoding = 'cp1252')
            # Remove space after comma, time portion of tSiteStartDate, and remove opening and closing square
            # brackets and commas
            dirtyStr = str(dirtyStr.split('\r\n')).replace("', '", ",").replace(" 00:00:00", "")[2:-3] + '\n'
            # Replace strings "', "" "", "" and "", '" with just a comma
            dirtyStr = dirtyStr.replace("', \"", ",").replace("\", \"", ",").replace("\", '", ",")
            dirtyStr = dirtyStr.replace("\\", "")                # Remove backslashes
            #debugFile.write('After: dirtyStr = ' + dirtyStr + '\n')

            dirtyFile.write(dirtyStr)                            # Write the line to the dirty file

        dirtyFile.close()
        #debugFile.close()

**main:_** Define folders for clean and dirty files, the input file, and other parameters, then generate the clean and dirty files.

In [18]:
if __name__ == "__main__":
    # Initialise file and folder names
    sliceBy             = 'Month'   # Value 'Week' means slice the input data by week number,
                                    # 'Fortnight' means slice the input data by fortnight number,
                                    # and 'Month' to slice by month
    baseFolder          = '<pathname_to_base_folder>'
    cleanName           = 'clean.By' + sliceBy
    dirtyName           = 'dirty.By' + sliceBy
    inputFile           = '<radiotherapy_treatment_filename>.csv'
    cleanPath           = baseFolder + cleanName
    dirtyPath           = baseFolder + dirtyName
    cleanFileNamePrefix = 'Radiotherapy_' + cleanName + '_'
    dirtyFileNamePrefix = 'Radiotherapy_' + dirtyName + '_'
    typoEveryNthChar    = 5                    # Swap every nth character with one adjacent on the QWERTY keyboard
    errorMagnitudes     = [1, 5, 10, 20, 30, 40, 50, 60, 70, 80]   # % of records with a synthetic error in a file
    
    inputPath = baseFolder + inputFile
    inputPathObj = Path(inputPath)
    
    if inputPathObj.exists():                  # If input file exists
        create_folder(baseFolder, cleanName)   # Create a new empty clean folder
        create_folder(baseFolder, dirtyName)   # Create a new empty dirty folder

        # Define all columns in the input file, specifying to read_csv that all are imported as type "object"
        dtypes = {
            'EpisodeID': 'object',
            'tSiteStartDate': 'object',
            'tSiteEndDate': 'object',
            'PatientID': 'object',
            'CaseID': 'object',
            'tMedicareNumber': 'object',
            'tMRN': 'object',
            'GroupID': 'object',
            'PatientAreaUniqueIdentifier': 'object',
            'tFirstGivenName': 'object',
            'tSecondGivenName': 'object',
            'tSurname': 'object',
            'tSex': 'object',
            'tDateOfBirth': 'object',
            'tURAStreet': 'object',
            'tURASuburb': 'object',
            'tURAPostcode': 'object',
            'tURAState': 'object',
            'tIndigenousStatus': 'object',
            'tAMO.AHPRA.RegistrationNumberOfTreatingDoctor': 'object',
            'tTreatingDoctorName': 'object',
            'tHospitalFacilityIdentifier': 'object',
            'tCancerDepartmentIdentifier': 'object',
            'FacilityType': 'object',
            'tDateOfPrimaryDiagnosis': 'object',
            'tPrimarySiteOfCancerCode': 'object',
            'tCancerBestBasisOfDiagnosis': 'object',
            'tLateralityOfPrimaryCancer': 'object',
            'tHistopathologicalGrade': 'object',
            'tMorphologyOfCancerCode': 'object',
            'tTStage': 'object',
            'tNStage': 'object',
            'tMStage': 'object',
            'tTNMStageGroup': 'object',
            'DerivedTNMStageGroup': 'object',
            'FinalTNMStageGroup': 'object',
            'TNMStagingGroupBasis': 'object',
            'tPSAScore': 'object',
            'tIntentionOfTreatment': 'object',
            'tRadiotherapyType': 'object',
            'tCourseName': 'object',
            'tPriorityCode': 'object',
            'tPrescribedDose': 'object',
            'tActualDose': 'object',
            'tPrescribedFractions': 'object',
            'tActualFractions': 'object',
            'tDateOfReferralToCancerSpecialist': 'object',
            'tReadyForCareDate': 'object',
            'tDateOfFirstConsultationWithCancerSpecialist': 'object',
            'tDateOfMultidisciplinaryTeamConsultation': 'object',
            'tDateOfReferralToPalliativeCare': 'object',
            'tPerformanceStatusDate': 'object',
            'tPerformanceStatus': 'object',
            'tTreatmentStartDate': 'object',
            'tTreatmentEndDate': 'object',
            'LHDCode': 'object',
            'TypeOfBatch': 'object',
            'ImportType': 'object',
            'CreatedNotificationId': 'object',
            'MessageInErrorFile.Batch.Review': 'object',
            'ProgER': 'object',
            'Progher2': 'object',
            'Progher2method': 'object',
            'Progher2method2': 'object',
            'Progher2status': 'object',
            'Progher2status2': 'object',
            'ProgPR': 'object',
            'ProgSummary': 'object',
            'tSource': 'object',
            'tLHDName': 'object',
            'SectorID': 'object',
            'tFacilityName': 'object',
            'TreatmentYear': 'object',
            'Snapshot': 'object',
            'NotificationMethod': 'object',
            'AgeAtTreatment': 'object',
            'NotificationType': 'object',
            'HospitalName': 'object',
            'PeerGroup2016': 'object',
            'PeerGroupName': 'object',
            'HEROIdentifier': 'object',
            'StateExcelFile': 'object',
            'PatientID.Derived': 'object',
            'RecordSource': 'object',
            'LoadDateTime': 'object',
            'TaskInstanceID': 'object',
            'Longitude': 'object',
            'Latitude': 'object',
            'MeshBlockCode2016': 'object',
            'MeshBlockCode2011': 'object',
            'CDCode1986': 'object',
            'CDCode1991': 'object',
            'CDCode1996': 'object',
            'CDCode2001': 'object',
            'CDCode2006': 'object',
            'LGACode2011': 'object',
            'LGACode2016': 'object',
            'LHDCode2010': 'object',
            'SLACode2011': 'object',
            'ValidityFlag': 'object',
            'Reliability': 'object',
            'GIFFlags': 'object',
            'AmendedFlag': 'object',
            'tReturnAddressLine': 'object',
            'tReturnLocalityName': 'object',
            'tReturnStateName': 'object',
            'tReturnPostCode': 'object',
            'AddressID.Derived': 'object',
            'AddressID': 'object',
            'CaseID.Derived': 'object',
            'RecordID': 'object',
            'tSiteName': 'object',
            'SiteNameBoost': 'object',
            'SiteNameLymphNode': 'object',
            'tRadiotherapyInformationSystemSiteName': 'object',
            'tNotifiableCancer': 'object',
            'LastDigit0': 'object',
            'LastDigit': 'object',
            'First3PrimaryICD10': 'object'}

        parse_dates = ['tSiteStartDate']   # the temporal column to be used to split into smaller files

        # Open the input file for reading
        df = pd.read_csv(inputPath, sep = ',', dtype = dtypes, parse_dates = parse_dates, encoding = 'cp1252')
        
        # For each continuous numeric column, we need to calculate the mean over the whole dataset, so that we can
        # use the mean to add Gaussian noise to a dirty file if all columns in a clean file are null.
        # The below column numbers: tPSAScore, tPrescribedDose, tActualDose, tPrescribedFractions, tActualFractions
        continuousNumericColumns = [37, 42, 43, 44, 45]
        continuousNumericMeans = {};
        for errorColumn in continuousNumericColumns:
            meanContinuousNumericValue = pd.to_numeric(df.iloc[:, errorColumn]).mean()
            # Convert column number to string because dictionary key cannot be numeric
            continuousNumericMeans[str(errorColumn)] = meanContinuousNumericValue

        generate_clean_files(df, sliceBy, cleanFileNamePrefix, cleanPath)

        generate_dirty_files(cleanPath, cleanName, dirtyPath, dirtyName, dtypes, parse_dates, errorMagnitudes,
                             typoEveryNthChar, continuousNumericMeans, continuousNumericColumns)
    else:
        print('Input file "' + inputPath + '" does not exist')
        
    print("Finished")

Finished
